# 1. annotate_mutations_all_modified

Preprocessing based on `annotate_mutations_all_modified.py` module in [MuAt model](https://github.com/primasanjaya/muat-github/blob/1f91c38d00d2f2156df9c2cb0f4e21dba673cf85/preprocessing/dmm/annotate_mutations_all_modified.py#L639). This notebook is utilised to just see how the preprocessing happens step by step.

In [1]:
import pandas as pd
import numpy as np

import argparse
import datetime
import glob
import gzip
import itertools
from natsort import natsort_keygen
import os
import shutil
import subprocess
import sys

from collections import deque

sys.path.insert(0, '/csc/epitkane/home/ahuttun/muat-github')

from preprocessing.dmm.annotate_mutations_all_modified import accepted_pos_h19, Variant, VariantReader, MAFReader, VCFReader, dna_comp_default
from preprocessing.dmm.util import openz, read_reference

To install swalign: pip install swalign


In [2]:
os.path.join(os.getcwd(), os.pardir, 'data','epipos')

'/csc/epitkane/projects/multimodal/notebooks/../data/epipos'

In [2]:
muat_github = '/csc/epitkane/home/ahuttun/muat-github'
mutation_codes_path = muat_github + '/extfile/mutation_codes_sv.tsv'  

In [3]:
mutation_table = pd.read_csv(mutation_codes_path, sep='\t')
mutation_table

,A A A
0,A C !
1,A G @
2,A T #
3,A N N
4,A - 1
5,C A $
6,C C C
7,C G %
8,C T ^
9,C N N


In [4]:
del mutation_table

In [5]:
mutation_codings = mutation_codes_path 
predict_filepath = muat_github + '/extfile/example_for_alldata_prediction_gz.tsv'
reference_h19 = muat_github +  '/ref/ref'
reference_h38 = muat_github +  '/ref/ref_hg38'
tmp_dir = muat_github + '/data/raw/temp/'
genomic_tracks = muat_github + '/preprocessing/genomic_tracks/h37/'
context = 8
report_interval = 10000
no_ref_preload = False
convert_hg38_hg19 = False
verbose = True
gel = False
info_column = False
no_filter = False
nope = True

In [6]:
def read_codes(fn):
    codes = {}
    rcodes = {}
    for s in open(fn):
        ref, alt, code = s.strip().split()
        if ref not in codes:
            codes[ref] = {}
        codes[ref][alt] = code
        rcodes[code] = (ref, alt)
    rcodes['N'] = ('N', 'N')  # ->N, N>-, A>N etc all map to N, make sure that 'N'=>'N>N'
    return codes, rcodes

In [7]:
def process_input(vr, o, sample_name, ref_genome, context,
                mutation_code, reverse_code):
    """A sweepline algorithm to insert mutations into the sequence flanking the focal mutation."""
    global warned_invalid_chrom
    prev_buf, next_buf = [], []
    i = 0
    n_var = n_flt = n_invalid = n_invalid_chrom = n_ok = 0
    for variant in vr:
        n_var += 1
        if report_interval > 0 and (n_var % report_interval) == 0:
            print('{} variants processed'.format(n_var))
        if no_ref_preload == False and variant.chrom not in ref_genome and variant.chrom != VariantReader.EOF:
            if warned_invalid_chrom == False:
                sys.stderr.write('Warning: a chromosome found in data not present in reference: {}\n'.format(variant.chrom))
                warned_invalid_chrom = True
            n_invalid_chrom += 1
            continue

        while len(next_buf) > 0 and (next_buf[0].chrom != variant.chrom or next_buf[0].pos < variant.pos - context):
            # if (len(next_buf) > 1):
            #     print(f'NEXT BUF {len(next_buf)}')
            # if (len(prev_buf) > 1):
            #     print(f'PREV BUF {len(prev_buf)}')    
            while len(prev_buf) > 0 and prev_buf[0].pos < next_buf[0].pos - context:
                prev_buf.pop(0)
## Following line 

            ctx = get_context(next_buf[0], prev_buf, next_buf, ref_genome,
                            mutation_code, reverse_code)
            if ctx is not None:
                # mutation not in the end of the chromosome and has full-len context
                next_buf[0].seq = ctx
                o.write(str(next_buf[0]) + '\n')
                n_ok += 1
            else:
                n_invalid += 1
            prev_buf.append(next_buf.pop(0)) # here next buf is popped out
        if len(prev_buf) > 0 and prev_buf[0].chrom != variant.chrom:
            prev_buf = []
        if variant.sample_id is None:
            variant.sample_id = sample_name   # id specific on per-file basis
        next_buf.append(variant)
    n_var -= 1  # remove terminator
    if verbose:
        n_all = vr.get_n_accepted() + vr.get_n_filtered()
        sys.stderr.write('{}/{} processed variants, {} filtered, {} invalid, {} missing chromosome\n'.\
            format(n_ok, n_all, vr.get_n_filtered(), n_invalid, n_invalid_chrom))
        sys.stderr.flush()

In [8]:
def open_stream(gel, tmp_dir, fn):
    if gel:
        sample_name = fn[:-7]
        sample_name = sample_name.split('/')
        sample_name = sample_name[0] + '_'.join(sample_name[1:])

        with gzip.open(fn, 'rb') as f_in:
            with open(tmp_dir + sample_name + '.vcf', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
            
            f = open(tmp_dir + sample_name + '.vcf')
    else:
        if fn.endswith('.gz'):
            #f = gzip.open(fn)
            #sample_name = os.path.basename(fn).split('.')[0]
            sample_name = os.path.basename(fn).split('/')[0]
            sample_name = sample_name[:-7]

            with gzip.open(fn, 'rb') as f_in:
                with open(tmp_dir + sample_name + '.vcf', 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

            f = open(tmp_dir + sample_name + '.vcf')
        else:
            f = open(fn)
            sample_name = os.path.basename(fn).split('.')[0]
        assert(('.maf' in fn and '.vcf' in fn) == False)  # filenames should specify input type unambigiously
    return f, sample_name

In [9]:
def get_reader(f, no_filter, info_column, type_snvs=False):
    if '.maf' in f.name:
        vr = MAFReader(f=f, pass_only=(no_filter == False), type_snvs=type_snvs,
                       extra_columns=info_column)
    elif '.vcf' in f.name:
        vr = VCFReader(f=f, pass_only=(no_filter == False), type_snvs=type_snvs)
    else:
        raise Exception('Unsupported file type: {}\n'.format(f.name))
    return vr

In [10]:
def get_context(v, prev_buf, next_buf, ref_genome,
                mutation_code, reverse_code):
    """Retrieve sequence context around the focal variant v, incorporate surrounding variants into
    the sequence."""
#    chrom, pos, fref, falt, vtype, _ = mut  # discard sample_id
    assert(context & (context - 1) == 0)
    flank = (context * 2) // 2 - 1 # 15
#    print('get_context', chrom, pos, fref, falt, args.context)
    if v.pos - flank - 1 < 0 or \
        (not no_ref_preload and v.pos + flank >= len(ref_genome[v.chrom])):
        return None
    if no_ref_preload:
        seq = subprocess.check_output(['samtools', 'faidx', reference,
                                      '{}:{}-{}'.format(v.chrom, v.pos - flank,
                                       v.pos + flank)])
        seq = ''.join(seq.decode().split('\n')[1:])
    else:
        seq = ref_genome[v.chrom][v.pos - flank - 1:v.pos + flank]
#    print('seqlen', len(seq))
    seq = list(seq)
    fpos = len(seq) // 2  # position of the focal mutation
    #for c2, p2, r2, a2, vt2, _ in itertools.chain(prev_buf, next_buf):
    #print(f'prev_buf: {prev_buf}')
    #print(f'next_buf: {next_buf}')
    for v2 in itertools.chain(prev_buf, next_buf):
        '''
        if len(prev_buf) > 0:
            print(f'prev_buf: {prev_buf}')
            print(f'next_buf: {next_buf}')
            print(f'V2: {v2}')
        '''
        if v.pos != v2.pos:
            print(f'V: {v}')
            print(f'V2: {v2}')
        if nope:
            if v.pos != v2.pos or (v.pos == v2.pos and (v.ref != v2.ref or v.alt != v2.alt)):
                continue
        # make sure that the mutation stays in the middle of the sequence!
        assert(v2.chrom == v.chrom)
        tp = v2.pos - v.pos + flank
        if tp < 0 or tp >= len(seq):
            continue
#        print(c2, p2, r2, a2, vt2, len(r2), len(a2), len(seq))
        if v2.vtype == Variant.SNV:
            seq[tp] = mutation_code[v2.ref][v2.alt]
        elif v2.vtype == Variant.DEL:
            for i, dc in enumerate(v2.ref):
#                    print('DEL', i, dc, mutation_code[r2[i + 1]]['-'])
                seq[tp] = mutation_code[dc]['-']
                tp += 1
                if tp == len(seq):
                    break
            if v.pos == v2.pos:
#                    print('ADJ, del', fpos, (len(r2) - 1) / 2)
                fpos += len(v2.ref) // 2  # adjust to the deletion midpoint
        elif v2.vtype == Variant.INS:
            seq[tp] = seq[tp] + ''.join([mutation_code['-'][ic] for ic in v2.alt])
            if v2.pos < v.pos:      # adjust to the insertion midpoint
                # v2 is before focal variant - increment position by insertion length
                fpos += len(v2.alt)
            elif v2.pos == v.pos:
                # v2 is the focal variant - increment position by half of insertion length
                fpos += int(np.ceil(1.0 * len(v2.alt) / 2))
        elif v2.vtype == Variant.COMPLEX:
            if support_complex:
                m = align(v2.ref, v2.alt, mutation_code)  # determine mutation sequence
                if len(m) + tp >= len(seq):  # too long allele to fit into the context; increase context length
                    return None
                for i in range(len(m)):  # insert mutation sequence into original
                    seq[tp] = m[i]
                    tp += 1
                n_bp_diff = len(v2.alt) - len(v2.ref)
                if n_bp_diff > 0: # inserted bases? add to the end of the block, insertions are unrolled below
                    seq[tp - 1] = seq[tp - 1] + ''.join(v2.alt[len(v2.ref):])
                # we need to adjust the midpoint according to whether block is before or at the current midpoint
                if v2.pos < v.pos:
                    fpos += max(0, n_bp_diff)
                elif v2.pos == v.pos:
                    fpos += (len(m) - 1) // 2
        elif v2.vtype in Variant.MEI_TYPES:
            seq[tp] = seq[tp] + mutation_code['-'][v2.vtype]
            if v2.pos <= v.pos:  # handle SV breakpoints as insertions
                fpos += 1
        elif v2.vtype in Variant.SV_TYPES:
            seq[tp] = seq[tp] + mutation_code['-'][v2.vtype]
            if v2.pos <= v.pos:  # handle SV breakpoints as insertions
                fpos += 1
        elif v2.vtype == Variant.NOM:
            pass  # no mutation, do nothing (a negative datapoint)
        else:
            raise Exception('Unknown variant type: {}'.format(v2))
    # unroll any insertions and deletions (list of lists -> list)
    seq = [x for sl in list(map(lambda x: list(x), seq)) for x in sl]
    #print('seq2', seq)
    n = len(seq)
    # reverse complement the sequence if the reference base of the substitution is not C or T,
    # or the first inserted/deleted base is not C or T.
    # we transform both nucleotides and mutations here
#    print('UNRL fpos={}, seq={}, f="{}", seqlen={}'.format(fpos, ''.join(seq), seq[fpos], len(seq)))
    lfref, lfalt = len(v.ref), len(v.alt)
    if (lfref == 1 and lfalt == 1 and v.ref in 'AG') or \
       ((v.alt not in Variant.SV_TYPES) and (v.alt not in Variant.MEI_TYPES) and \
            ((lfref > 1 and v.ref[1] in 'AG') or (lfalt > 1 and v.alt[1]))):
        # dna_comp_default returns the input character for non-DNA characters (SV breakpoints)
        seq = [mutation_code[dna_comp_default(reverse_code.get(x)[0])]\
            [dna_comp_default(reverse_code.get(x)[1])] for x in seq][::-1]
        fpos = n - fpos - 1
#        print('REVC', fref, falt, 'fpos={}, seq={}, f="{}", seqlen={}'.format(fpos, ''.join(seq), seq[fpos], len(seq)))
    target_len = 2**int(np.floor(np.log2(context)))
    # trim sequence to length 2^n for max possible n
    #target_len = 2**int(np.floor(np.log2(n)))
    #trim = (n - target_len) / 2.0
    seq = ''.join(seq[max(0, fpos - int(np.floor(target_len / 2))):min(n, fpos + int(np.ceil(target_len / 2)))])
#    print('TRIM seqlen={}, tgtlen={}, seq={}, mid="{}"'.format(len(seq), target_len, ''.join(seq), seq[len(seq) // 2]))
    if len(seq) != target_len:
        return None
    return seq[3:6]


Going through the `func_annotate_mutation_all_modified` in `annotate_mutations_all_modified.py`

In [11]:
mutation_code, reverse_mutation_code = read_codes(mutation_codings)

global warned_invalid_chrom
warned_invalid_chrom = False

fns  = pd.read_csv(predict_filepath,sep='\t',low_memory=False)['path'].to_list()

###########
# Added for notebook
###########
for i in range(len(fns)):
    fns[i] = muat_github + fns[i][1:]
###########
n_missing = 0

for fn in fns:
    if os.path.exists(fn) == False:
        sys.stderr.write('Input file {} not found\n'.format(fn))
        n_missing += 1
if n_missing > 0:
    sys.exit(1)

print('{} input files found'.format(len(fns)))
if len(fns) == 0:
        sys.exit(1)


5 input files found


In [12]:
print(mutation_code)

{'A': {'A': 'A', 'C': '!', 'G': '@', 'T': '#', 'N': 'N', '-': '1'}, 'C': {'A': '$', 'C': 'C', 'G': '%', 'T': '^', 'N': 'N', '-': '2'}, 'G': {'A': '&', 'C': '*', 'G': 'G', 'T': '~', 'N': 'N', '-': '3'}, 'T': {'A': ':', 'C': ';', 'G': '?', 'T': 'T', 'N': 'N', '-': '4'}, 'N': {'N': 'N', '-': 'N'}, '-': {'A': '5', 'C': '6', 'G': '7', 'T': '8', 'N': 'N', 'SV_DEL': 'D', 'SV_DUP': 'P', 'SV_INV': 'I', 'SV_BND': 'B'}}


In [13]:
if convert_hg38_hg19:
    print('Reading reference h38... ')
    reference_38 = read_reference(reference_h38, verbose)   

    print('Reading reference h19... ')
    reference_19 = read_reference(reference_h19, verbose)

else:    
    print('Reading reference h19... ')
    reference_19 = read_reference(reference_h19, verbose)

1 

Reading reference h19... 


2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 X Y MT GL000207.1 GL000226.1 GL000229.1 GL000231.1 GL000210.1 GL000239.1 GL000235.1 GL000201.1 GL000247.1 GL000245.1 GL000197.1 GL000203.1 GL000246.1 GL000249.1 GL000196.1 GL000248.1 GL000244.1 GL000238.1 GL000202.1 GL000234.1 GL000232.1 GL000206.1 GL000240.1 GL000236.1 GL000241.1 GL000243.1 GL000242.1 GL000230.1 GL000237.1 GL000233.1 GL000204.1 GL000198.1 GL000208.1 GL000191.1 GL000227.1 GL000228.1 GL000214.1 GL000221.1 GL000209.1 GL000218.1 GL000220.1 GL000213.1 GL000211.1 GL000199.1 GL000217.1 GL000216.1 GL000215.1 GL000205.1 GL000219.1 GL000224.1 GL000223.1 GL000195.1 GL000212.1 GL000222.1 GL000200.1 GL000193.1 GL000194.1 GL000225.1 GL000192.1 NC_007605 hs37d5  done.


In [16]:
try:
    os.makedirs(os.path.dirname(tmp_dir))
except:
    pass

Following loop is loop for all the cells below that in the real function 

In [42]:
all_error_file = []
all_succeed_file = []
for i, fn in enumerate(fns):
    #try:
    #check VCF version:

    f = gzip.open(fn, 'rt')
    variable = f.readline()

    if variable == '##fileformat=VCFv4.2\n':
        version = '4.2'
    elif variable == '##fileformat=VCFv4.1\n':
        version = '4.1'
    else:
        print('current MuAt version cant process the VCF file version')
    #try:

    if gel:
        sample_name = fn[:-7]
        sample_name = sample_name.split('/')
        sample_name = sample_name[0] + '_'.join(sample_name[1:])
    else:
        get_ext = fn[-4:]
        if get_ext == '.vcf':
            sample_name = fn[:-4]
        else:
            sample_name = fn[:-7]
        #pdb.set_trace()
        sample_name = sample_name.split('/')
        sample_name = sample_name[-1]
    
    print(sample_name)
    process = []

    if version == '4.1':
        output_file = tmp_dir + sample_name + '.tsv.gz'
        o = gzip.open(output_file, 'wt')

        f, sample_name_2 = open_stream(gel, tmp_dir, fn)

        digits = int(np.ceil(np.log10(len(fns))))
        fmt = '{:' + str(digits) + 'd}/{:' + str(digits) + 'd} {}: '
        if info_column:
            infotag = '\t{}'.format('\t'.join(map(str.lower, info_column)))
        else:
            infotag = ''
        print('Writing mutation sequences...')

        o.write('chrom\tpos\tref\talt\tsample\tseq{}\n'.format(infotag))

        print(fmt.format(i + 1, len(fns), sample_name))
        vr = get_reader(f, no_filter, info_column)
        if convert_hg38_hg19:
            process_input(vr, o, sample_name_2, reference_38, context,mutation_code, reverse_mutation_code)
        else:
            process_input(vr, o, sample_name_2, reference_19, context,mutation_code, reverse_mutation_code)
        f.close()
        o.close()
        print('Output written to {}'.format(output_file))
        #pdb.set_trace()
        #natural sort motif
        pd_motif = pd.read_csv(output_file,sep='\t',low_memory=False,compression='gzip') 
        mutation = len(pd_motif)
        pd_motif  = pd_motif.sort_values(by=['chrom','pos'],key=natsort_keygen())
        pd_motif.to_csv(output_file,sep='\t',index=False, compression="gzip")
        #end motif
        process.append('motif')

0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv
Writing mutation sequences...
1/5 0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv: 
V: 1	183373645	G	A	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V2: 1	183373652	T	A	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V: 1	183373652	T	A	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V2: 1	183373645	G	A	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	T^A
V: 12	43189276	G	T	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V2: 12	43189277	G	T	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V: 12	43189277	G	T	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V2: 12	43189276	G	T	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	C$A
V: 13	55521253	G	A	0a6be23a-d5a0-4e95-ada2-a61b2b5

4219/4219 processed variants, 0 filtered, 0 invalid, 0 missing chromosome


V: 7	76779950	G	T	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V2: 7	76779951	G	T	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V: 7	76779951	G	T	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V2: 7	76779950	G	T	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	C$A
V: X	66195676	G	A	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V2: X	66195681	C	G	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V: X	66195681	C	G	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V2: X	66195676	G	A	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	C^T
V: X	66196097	A	G	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V2: X	66196101	G	C	0a6be23a-d5a0-4e95-ada2-a61b2b5d9485.consensus.20160830.somatic.snv_mnv	None
V: X	66196101	G	C	0a6be23a-d5a0-4e95-ada2-a61b2

V: 2	81853014	G	T	0a9c9db0-c623-11e3-bf01-24c6515278c0.consensus.20160830.somatic.snv_mnv	None
V2: 2	81853015	G	T	0a9c9db0-c623-11e3-bf01-24c6515278c0.consensus.20160830.somatic.snv_mnv	None
V: 2	81853015	G	T	0a9c9db0-c623-11e3-bf01-24c6515278c0.consensus.20160830.somatic.snv_mnv	None
V2: 2	81853014	G	T	0a9c9db0-c623-11e3-bf01-24c6515278c0.consensus.20160830.somatic.snv_mnv	C$A
V: 2	103617743	G	C	0a9c9db0-c623-11e3-bf01-24c6515278c0.consensus.20160830.somatic.snv_mnv	None
V2: 2	103617744	G	T	0a9c9db0-c623-11e3-bf01-24c6515278c0.consensus.20160830.somatic.snv_mnv	None
V: 2	103617744	G	T	0a9c9db0-c623-11e3-bf01-24c6515278c0.consensus.20160830.somatic.snv_mnv	None
V2: 2	103617743	G	C	0a9c9db0-c623-11e3-bf01-24c6515278c0.consensus.20160830.somatic.snv_mnv	C%T
V: 2	153928928	C	A	0a9c9db0-c623-11e3-bf01-24c6515278c0.consensus.20160830.somatic.snv_mnv	None
V2: 2	153928929	C	T	0a9c9db0-c623-11e3-bf01-24c6515278c0.consensus.20160830.somatic.snv_mnv	None
V: 2	153928929	C	T	0a9c9db0-c623-11e3-bf0

11130/11130 processed variants, 0 filtered, 0 invalid, 0 missing chromosome


...

4051/4051 processed variants, 0 filtered, 0 invalid, 0 missing chromosome


00b9d0e6-69dc-4345-bffd-ce32880c8eef.consensus.20160830.somatic.snv_mnv
Writing mutation sequences...
4/5 00b9d0e6-69dc-4345-bffd-ce32880c8eef.consensus.20160830.somatic.snv_mnv: 
V: 1	202789605	T	G	00b9d0e6-69dc-4345-bffd-ce32880c8eef.consensus.20160830.somatic.snv_mnv	None
V2: 1	202789606	T	A	00b9d0e6-69dc-4345-bffd-ce32880c8eef.consensus.20160830.somatic.snv_mnv	None
V: 1	202789606	T	A	00b9d0e6-69dc-4345-bffd-ce32880c8eef.consensus.20160830.somatic.snv_mnv	None
V2: 1	202789605	T	G	00b9d0e6-69dc-4345-bffd-ce32880c8eef.consensus.20160830.somatic.snv_mnv	T?T
V: 10	81745792	T	G	00b9d0e6-69dc-4345-bffd-ce32880c8eef.consensus.20160830.somatic.snv_mnv	None
V2: 10	81745793	T	A	00b9d0e6-69dc-4345-bffd-ce32880c8eef.consensus.20160830.somatic.snv_mnv	None
V: 10	81745793	T	A	00b9d0e6-69dc-4345-bffd-ce32880c8eef.consensus.20160830.somatic.snv_mnv	None
V2: 10	81745792	T	G	00b9d0e6-69dc-4345-bffd-ce32880c8eef.consensus.20160830.somatic.snv_mnv	C?T
V: 13	36673122	T	G	00b9d0e6-69dc-4345-bffd-ce32880

00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv
Writing mutation sequences...
5/5 00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv: 
V: 1	3279474	G	T	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V2: 1	3279477	C	A	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V: 1	3279477	C	A	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V2: 1	3279474	G	T	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	A$C
V: 1	3564120	G	A	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V2: 1	3564121	A	G	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V: 1	3564121	A	G	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V2: 1	3564120	G	A	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	T^C
V: 1	6742567	C	T	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20

9478/9478 processed variants, 0 filtered, 0 invalid, 0 missing chromosome


V: 20	61799891	G	A	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V2: 20	61799892	A	T	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V: 20	61799892	A	T	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V2: 20	61799891	G	A	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	T^C
V: 21	11149808	C	A	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V2: 21	11149809	A	C	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V: 21	11149809	A	C	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V2: 21	11149808	C	A	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	A$A
V: 22	38881964	G	A	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V2: 22	38881967	A	G	00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv	None
V: 22	38881967	A	G	00db1b95-8ca3-4cc4

In [18]:
pd_motif.head()

,chrom,pos,ref,alt,sample,seq
0,1,1015594,C,G,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,G%C
1,1,1866371,C,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,A$A
2,1,1921712,C,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,C$G
3,1,2049858,G,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,G^T
4,1,2357842,C,T,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,C^G


**NOTE** In the function `process_input()`, the tsv files are created and the motif sequences are created there!

skip lines 760-799 i.e, the case `convert_hg38_hg19 == True` is skipped for now. **Continue from line 800**

In [19]:
mut_liftover = 0
process.append('no-liftover')

GC content

In [20]:
#next gc content

input_gc = output_file
output_gc = tmp_dir + sample_name + '.gc.tsv.gz' # New files are created
label = 'gc1kb'
verbose = True
window = 1001
batch_size = 100000

o = openz(output_gc, 'wt')

In [21]:
hdr = openz(input_gc).readline().strip().decode("utf-8").split('\t')
n_cols = len(hdr)
sys.stderr.write('{} columns in input\n'.format(n_cols))
if hdr[0] == 'chrom':
    sys.stderr.write('Header present\n')
    #pdb.set_trace()
    o.write('{}\t{}\n'.format('\t'.join(hdr), label))
else:
    sys.stderr.write('Header absent\n')
    hdr = None

6 columns in input
Header present


In [22]:
n_mut = 0
then = datetime.datetime.now()
with openz(input_gc) as f:
    if hdr:
        f.readline()
    cchrom = None
    for s in f:
        v = s.strip().decode("utf-8").split('\t')
        chrom, pos = v[0], int(v[1])
        if cchrom != chrom:
            cchrom = chrom
            cpos = max(0, pos - window / 2)
            mpos = min(len(reference_19[chrom]) - 1, cpos + window)
            cpos = round(cpos)
            mpos = round(mpos)
            buf = deque(reference_19[chrom][cpos:mpos])
            gc = sum([1 for c in buf if c == 'C' or c == 'G'])
            at = sum([1 for c in buf if c == 'A' or c == 'T'])
        else:
            cpos2 = max(0, pos - window / 2)
            cdiff = cpos2 - cpos
            if cdiff > 0:
                if cdiff < window:
                    # incremental update of buffer
                    for _ in range(round(cdiff)):
                        remove = buf.popleft()
                        if remove == 'C' or remove == 'G':
                            gc -= 1
                        elif remove == 'A' or remove == 'T':
                            at -= 1
                    insert = reference_19[cchrom][round(cpos+window):round(cpos+window+cdiff)]
                    gc += sum([1 for c in insert if c == 'C' or c == 'G'])
                    at += sum([1 for c in insert if c == 'A' or c == 'T'])
                    buf.extend(insert)
                else:
                    # reinit buffer at cpos2
                    mpos = min(len(reference_19[chrom]) - 1, cpos2 + window)
                    buf = deque(reference_19[chrom][round(cpos2):round(mpos)])
                    gc = sum([1 for c in buf if c == 'C' or c == 'G'])
                    at = sum([1 for c in buf if c == 'A' or c == 'T'])
                cpos = cpos2
        try:
            gc_ratio = 1.0 * gc / (gc + at)
        except:
            gc_ratio = 0

        o.write('{}\t{}\n'.format(s.strip().decode("utf-8"), gc_ratio))

        n_mut += 1
        if verbose and (n_mut % batch_size) == 0:
            now = datetime.datetime.now()
            td = now - then
            sys.stderr.write('{}: {} mutations. {}/mutation\n'.format(now, n_mut, td / batch_size))
            then = now
o.close()

In [23]:
pd_sort = pd.read_csv(output_gc,sep='\t',low_memory=False,compression="gzip")
#remove nan genic
pd_sort = pd_sort[~pd_sort['gc1kb'].isna()]
pd_sort['chrom'] = pd_sort['chrom'].astype('string')
pd_sort = pd_sort.loc[pd_sort['chrom'].isin(accepted_pos_h19)]
pd_sort = pd_sort.sort_values(by=['chrom','pos'],key=natsort_keygen())
pd_sort.to_csv(output_gc,sep='\t',index=False, compression="gzip")

process.append('gc')

#############
pd_sort.head() 
#############

,chrom,pos,ref,alt,sample,seq,gc1kb
0,1,1015594,C,G,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,G%C,0.674000
1,1,1866371,C,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,A$A,0.492016
2,1,1921712,C,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,C$G,0.446000
3,1,2049858,G,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,G^T,0.460000
4,1,2357842,C,T,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,C^G,0.617000


In [24]:
del pd_sort

Genic region 

In [25]:
output_genic = tmp_dir + sample_name + '.gc.genic.tsv.gz'
syntax_genic = muat_github + '/preprocessing/dmm/annotate_mutations_with_bed.sh \
' + output_gc + ' \
' + genomic_tracks + 'Homo_sapiens.GRCh37.87.genic.genomic.bed.gz \
'+ output_genic + ' \
genic'
syntax_genic
subprocess.run(syntax_genic, shell=True)


Extracting header from /csc/epitkane/home/ahuttun/muat-github/data/raw/temp/00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv.gc.tsv.gz ...
Annotating /csc/epitkane/home/ahuttun/muat-github/data/raw/temp/00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv.gc.tsv.gz with /csc/epitkane/home/ahuttun/muat-github/preprocessing/genomic_tracks/h37/Homo_sapiens.GRCh37.87.genic.genomic.bed.gz and writing to /csc/epitkane/home/ahuttun/muat-github/data/raw/temp/00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv.gc.genic.tsv.gz ...
Tue 16 Apr 11:03:38 EEST 2024
All done
Tue 16 Apr 11:03:39 EEST 2024


CompletedProcess(args='/csc/epitkane/home/ahuttun/muat-github/preprocessing/dmm/annotate_mutations_with_bed.sh /csc/epitkane/home/ahuttun/muat-github/data/raw/temp/00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv.gc.tsv.gz /csc/epitkane/home/ahuttun/muat-github/preprocessing/genomic_tracks/h37/Homo_sapiens.GRCh37.87.genic.genomic.bed.gz /csc/epitkane/home/ahuttun/muat-github/data/raw/temp/00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv.gc.genic.tsv.gz genic', returncode=0)

In [26]:
pd_sort = pd.read_csv(output_genic,sep='\t',low_memory=False,compression="gzip")
#remove nan genic
pd_sort = pd_sort[~pd_sort['genic'].isna()]
pd_sort['chrom'] = pd_sort['chrom'].astype('string')
pd_sort = pd_sort.loc[pd_sort['chrom'].isin(accepted_pos_h19)]
pd_sort = pd_sort.sort_values(by=['chrom', 'pos'],key=natsort_keygen())
pd_sort.to_csv(output_genic,sep='\t',index=False, compression="gzip")
process.append('genic')

#############
pd_sort.head()
#############

,chrom,pos,ref,alt,sample,seq,gc1kb,genic
0,1,1015594,C,G,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,G%C,0.674000,0.0
1,1,1866371,C,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,A$A,0.492016,1.0
2,1,1921712,C,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,C$G,0.446000,1.0
3,1,2049858,G,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,G^T,0.460000,1.0
4,1,2357842,C,T,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,C^G,0.617000,1.0


In [27]:
del pd_sort

Exon region 

In [28]:
output_exon = tmp_dir + sample_name + '.gc.genic.exonic.tsv.gz'

syntax_exonic = muat_github + '/preprocessing/dmm/annotate_mutations_with_bed.sh \
' + output_genic + ' \
' + genomic_tracks + 'Homo_sapiens.GRCh37.87.exons.genomic.bed.gz \
' + output_exon + ' \
exonic'
subprocess.run(syntax_exonic, shell=True)

pd_sort = pd.read_csv(output_exon,sep='\t',low_memory=False,compression="gzip") 
pd_sort = pd_sort[~pd_sort['exonic'].isna()]
pd_sort['chrom'] = pd_sort['chrom'].astype('string')
pd_sort = pd_sort.loc[pd_sort['chrom'].isin(accepted_pos_h19)]
pd_sort = pd_sort.sort_values(by=['chrom', 'pos'],key=natsort_keygen())
pd_sort.to_csv(output_exon,sep='\t',index=False, compression="gzip")

process.append('exonic')

#############
pd_sort.head()
#############

Extracting header from /csc/epitkane/home/ahuttun/muat-github/data/raw/temp/00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv.gc.genic.tsv.gz ...
Annotating /csc/epitkane/home/ahuttun/muat-github/data/raw/temp/00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv.gc.genic.tsv.gz with /csc/epitkane/home/ahuttun/muat-github/preprocessing/genomic_tracks/h37/Homo_sapiens.GRCh37.87.exons.genomic.bed.gz and writing to /csc/epitkane/home/ahuttun/muat-github/data/raw/temp/00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus.20160830.somatic.snv_mnv.gc.genic.exonic.tsv.gz ...
Tue 16 Apr 11:03:39 EEST 2024
All done
Tue 16 Apr 11:03:40 EEST 2024


,chrom,pos,ref,alt,sample,seq,gc1kb,genic,exonic
0,1,1015594,C,G,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,G%C,0.674000,0.0,0.0
1,1,1866371,C,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,A$A,0.492016,1.0,0.0
2,1,1921712,C,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,C$G,0.446000,1.0,0.0
3,1,2049858,G,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,G^T,0.460000,1.0,0.0
4,1,2357842,C,T,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,C^G,0.617000,1.0,0.0


In [29]:
del pd_sort

In [30]:
output_cs = tmp_dir + sample_name + '.gc.genic.exonic.cs.tsv.gz'
annotation = genomic_tracks + 'Homo_sapiens.GRCh37.87.transcript_directionality.bed.gz'

o = openz(output_cs, 'wt')

hdr = openz(output_exon).readline().strip().decode("utf-8").split('\t')
n_cols = len(hdr)
sys.stderr.write('{} columns in input\n'.format(n_cols))
if hdr[0] == 'chrom':
    sys.stderr.write('Header present\n')
    o.write('{}\tstrand\n'.format('\t'.join(hdr)))
else:
    sys.stderr.write('Header absent\n')
    hdr = None
sys.stderr.write('Reading reference: ')
#reference = read_reference(args.ref, verbose=True)
cmd = "bedmap --sweep-all --faster --delim '\t' --multidelim '\t' --echo --echo-map  <(gunzip -c {muts}|grep -v \"^chrom\"|awk 'BEGIN{{FS=OFS=\"\t\"}} {{$2 = $2 OFS $2+1}} 1') <(zcat {annot})".format(annot=annotation, muts=output_exon)
#pdb.set_trace()
p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, executable='/bin/bash')

9 columns in input
Header present
Reading reference: 

In [31]:
prev_chrom = prev_pos = None
seen_chroms = set()
n = 0
for s in p.stdout:
    v = s.strip().decode("utf-8").split('\t')
    # v is mutation bed + extra columns when the mutation overlaps with a transcript
    # extra columns: chrom,start,end,dirs where dirs is either 1) +, 2) -, 3) +,-
    n_pos = n_neg = 0 
    if len(v) == n_cols + 1:
        pass  # no overlap
    else:
        try:
            strands = v[n_cols + 1 + 3]  # +1 for extra bed END column, +3 to get the strand from [chrom, start, end, strand]
            if strands not in ['+', '-', '+;-']:
                raise Exception('Unknown strand directionality {} at \n{}'.format(strands, s))
            if strands == '+':
                n_pos = 1
            elif strands == '-':
                n_neg = 1
            else:
                n_pos = n_neg = 1
        except:
            pdb.set_trace()

#        n_pos = len(filter(lambda x: x == '+', strands))
#        n_neg = len(filter(lambda x: x == '-', strands))
#        st = None
    chrom, pos = v[0], int(v[1])
    if chrom != prev_chrom:
        if chrom in seen_chroms:
            sys.stderr.write('Input is not sorted (chromosome order): {}:{}\n'.format(chrom, pos))
            sys.exit(1)
        seen_chroms.add(chrom)
        prev_chrom = chrom
    else:
        if pos < prev_pos:
            sys.stderr.write('Input is not sorted (position order): {}:{}\n'.format(chrom, pos))
            sys.exit(1)
    prev_pos = pos
    ref, alt = v[3], v[4]
    #pdb.set_trace()
    base = reference_19[chrom][pos]
    if n_pos > 0:
        if n_neg > 0:
            st = '?'
        else:
            if base in ['C', 'T']:
                st = '+'
            else:
                st = '-'
    else:
        if n_neg > 0:
            if base in ['C', 'T']:
                st = '-'
            else:
                st = '+'
        else:
            st = '='
    o.write('{}\t{}\t{}\t{}\n'.format(chrom, pos, '\t'.join(v[3:n_cols + 1]), st))
    n += 1
    if (n % 1000000) == 0:
        sys.stdout.write('{}: {} mutations written\n'.format(datetime.datetime.now(), n))
o.flush()
o.close()

In [32]:
pd_sort = pd.read_csv(output_cs,sep='\t',low_memory=False,compression="gzip") 
pd_sort = pd_sort[~pd_sort['strand'].isna()]

pd_sort['chrom'] = pd_sort['chrom'].astype('string')
pd_sort = pd_sort.loc[pd_sort['chrom'].isin(accepted_pos_h19)]
pd_sort = pd_sort.sort_values(by=['chrom', 'pos'],key=natsort_keygen())
preprocessed_mutation = len(pd_sort)
pd_sort.to_csv(output_cs,sep='\t',index=False, compression="gzip")

process.append('strand')

#############
pd_sort.head()
#############

,chrom,pos,ref,alt,sample,seq,gc1kb,genic,exonic,strand
0,1,1015594,C,G,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,G%C,0.674000,0.0,0.0,=
1,1,1866371,C,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,A$A,0.492016,1.0,0.0,+
2,1,1921712,C,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,C$G,0.446000,1.0,0.0,+
3,1,2049858,G,A,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,G^T,0.460000,1.0,0.0,+
4,1,2357842,C,T,00db1b95-8ca3-4cc4-bb46-6b8c8019a7c7.consensus...,C^G,0.617000,1.0,0.0,-


In [33]:
del pd_sort

The lines **1040-1061** are skipped for now since convert_hg38_hg19 is False

In [34]:
if process == ['motif','no-liftover','gc','genic','exonic','strand']:
    tup_mut = [(mutation,mut_liftover,preprocessed_mutation)]
    pd_complete_mutation = pd.DataFrame(tup_mut)
    pd_complete_mutation.columns = ['original_mutation','liftover_mutation','preprocessed_mutation']
    pd_complete_mutation.to_csv(tmp_dir + sample_name + '_mutations.csv')
    #task complete

    try:
        os.remove(tmp_dir + sample_name + '.vcf')
    except:
        pass
    os.remove(tmp_dir + sample_name + '.tsv.gz')
    os.remove(tmp_dir + sample_name + '.gc.tsv.gz')
    os.remove(tmp_dir + sample_name + '.gc.genic.tsv.gz')
    os.remove(tmp_dir + sample_name + '.gc.genic.exonic.tsv.gz')
    all_succeed_file.append(fn)
else:
    all_files = glob.glob(tmp_dir + sample_name + '*')
    for i_files in range(len(all_files)):
        os.remove(all_files[i_files])

In [35]:
pd_complete_mutation.head()

,original_mutation,liftover_mutation,preprocessed_mutation
0,9478,0,9477
